<a href="https://colab.research.google.com/github/Seo-Jeong-Lee/tmp/blob/main/Baseline_codes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [16]:
import numpy as np
import random
import os
import math

from glob import glob
import pandas as pd
import cv2
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import torchvision.models as models
from torchvision import transforms

In [17]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameters Setting

Hyperparameter(하이퍼 파라미터) : 모델링할 때 사용자가 직접 세팅해주는 값

예를 들어, learning rate, support-vector-machine(SVM)에서의 c값, KNN에서의 K값 등이 있음

In [18]:
CFG = {
    'IMG_SIZE':128,
    'EPOCHS':10,
    'LEARNING_RATE':2e-3,
    'BATCH_SIZE':8,
    'SEED':41
}

## Fix RandomSeed

학습한 모델을 Reproduction 하기 위해 Seed 를 고정해야함

모델의 Reproduction(복제)이 필요한 경우 : *재현성(Reproducibility)* 유지가 중요

1. 수상자가 되어 코드의 정합성을 검증 받게 될 경우,

1. 경진대회 참가 도중 팀을 이루어 결과를 공유해야 되는 경우,

1. 논문을 작성하여 그 결과를 Reproduction 해야하는 경우 등

**재현성(Reproducibility) : 실험을 여러번 했을 때 똑같은 결과를 보장하는 것**

In [19]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True             #True 인 경우 cuDNN이 결정적 회선 알고리즘 만 사용하도록 함.
    torch.backends.cudnn.benchmark = True                 #True 인 경우 cuDNN이 다중 회선 알고리즘을 벤치마킹하고 가장 빠른 알고리즘을 선택하도록 함.

seed_everything(CFG['SEED']) # Seed 고정

**np.random.seed : 난수 발생을 위해서는 적절한 시드(seed)를 난수 발생기에 주어야 함. 만약 시드가 같다면 동일한 난수를 발생시키게 됨**

## Data Pre-processing

iterable : 한 번에 하나의 member를 반환할 수 있는 object.

ex) list, str, tuple 

In [24]:
def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    for case_name in os.listdir(data_dir):
        current_path = os.path.join(data_dir, case_name)
        if os.path.isdir(current_path):                                #os.path.isdir : 디렉토리의 유무를 확인하여 True 혹은 False를 반환하는 함수
            # get image path
            img_path_list.extend(glob(os.path.join(current_path, 'image', '*.jpg')))
            img_path_list.extend(glob(os.path.join(current_path, 'image', '*.png')))      #image path를 img_path_list에 저장하는 과정
             
            # get label
            label_df = pd.read_csv(current_path+'/label.csv')
            label_list.extend(label_df['leaf_weight'])                                    #label을 label_list에 저장하는 과정
                
    return img_path_list, label_list

def get_test_data(data_dir):
    # get image path
    img_path_list = glob(os.path.join(data_dir, 'image', '*.jpg'))
    img_path_list.extend(glob(os.path.join(data_dir, 'image', '*.png')))
    img_path_list.sort(key=lambda x:int(x.split('/')[-1].split('.')[0]))
    return img_path_list

extend()함수 : 삽입 대상의 리스트를 풀어서 각각의 엘리먼트로 확장(Extend)해 삽입함. append와 기능적으로는 유사함.

```
>>> a = [1,2,3]
>>> b = [4, 5]
>>> a.extend(b)
>>> a
[1, 2, 3, 4, 5]
```

glob() 함수 : 괄호 안에, 인자로 받은 패턴과 이름이 일치하는 모든 파일과 디렉터리의 리스트를 반환함. 

```
 from glob import glob
>>> glob('*.exe')               # 현재 디렉터리의 .exe 파일
['python.exe', 'pythonw.exe']
>>> glob('*.txt')               # 현재 디렉터리의 .txt 파일
['LICENSE.txt', 'NEWS.txt']
```
패턴을 그냥 *라고 주면 모든 파일과 디렉터리를 볼 수 있음

*img_path_list.extend(glob(os.path.join(current_path, 'image', '*.jpg')))

위의 코드를 이해하기 위해 다음의 링크(Answer#2) 참조함
https://discuss.dizzycoding.com/python-glob-multiple-filetypes/


****Competition Data 연동하기****

참조 : https://teki.tistory.com/m/29

In [30]:
#구글 드라이브에 open파일 업로드 후 구글드라이브와 Colab연동하기
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#위에서 만든 함수 get_train_data와 get_test_data를 이용하여 data 불러오기
all_img_path, all_label = get_train_data('./dataset/train')
test_img_path = get_test_data('./dataset/test')

## Train / Validation Split

In [15]:
# Train : Validation = 0.8 : 0.2 Split
train_len = int(len(all_img_path)*0.8)

train_img_path = all_img_path[:train_len]
train_label = all_label[:train_len]

vali_img_path = all_img_path[train_len:]
vali_label = all_label[train_len:]

NameError: ignored

## CustomDataset : Dataset을 상속 받아 Cusstom Dataset class 만들기

참조 : 

https://sanghyu.tistory.com/90

https://data-panic.tistory.com/21

https://didu-story.tistory.com/85

https://blog.promedius.ai/pytorch_dataloader_1/


In [ ]:
class CustomDataset(Dataset):   #부모클래스가 Dataset, 자식클래스는 CustomDataset이다. 자식클래스는 부모클래스로부터 상속받는다.
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):     #생성자 부분. 속성을 초기화하는 목적으로 사용. 
        self.transforms = transforms                                                     #메서드를 호출할 때 self 자리는 비워놓는다.(입력하지 않는다)
        self.train_mode = train_mode                                                     #이 클래스의 속성은 4개이다.(transforms, train_mode, img_path_list, label_list)          
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):                                                         
        img_path = self.img_path_list[index]   #앞서 만든 리스트(img_path_list)의 인덱스값을 참조해 해당 이미지를 연 다음, (4줄 아래와 이어짐)
        # Get image data
        image = cv2.imread(img_path)        
        if self.transforms is not None:     #transforms 가 None이 아닌 경우 : transforms를 실행할 경우
            image = self.transforms(image)  #tensor 자료형으로 바꾸어 이미지 전처리를 실행하는 구조이다.

        if self.train_mode: #train_mode = True라면,
            label = self.label_list[index]    #앞서 만든 리스트(label_list)의 인덱스값을 참조해 해당 label을 연다
            return image, label
        else:               #train_mode = False라면, 해당 이미지만을 연다
            return image
    
    def __len__(self):      #학습 데이터의 갯수를 반환하는 역할을 한다
        return len(self.img_path_list) 

**torchvision.transforms**를 이용하는 데이터 전처리 

: transforms에 속한 함수들을 Compose를 통해 묶어서 한번에 처리할 수 있다.

-> transforms의 여러 메서드들은 다음의 링크를 참조함:

https://mhko411.tistory.com/157

In [ ]:
train_transform = transforms.Compose([
                    transforms.ToTensor(),                                                 #PIL 이미지 또는 numpy.ndarray 형식의 이미지를 tensor 형식으로 변환한다.(0~255->0~1) 
                    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),                 #주어진 PIL 이미지를 입력한 size로 크기 조정을 한다.
                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))        #주어진 이미지를 mean, std의 값을 통해 정규화 한다.
                    ])

test_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                    ])

*주의할 점 : 항상 transforms.Normalize는 transforms.ToTensor 뒤에 와야한다*

참조 : https://89douner.tistory.com/299

In [ ]:
# Get Dataloader
train_dataset = CustomDataset(train_img_path, train_label, train_mode=True, transforms=train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

vali_dataset = CustomDataset(vali_img_path, vali_label, train_mode=True, transforms=test_transform)
vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

DataLoader는 일반적으로 샘플들을 mini batch로 전달하고, 

매 epoch마다 데이터를 다시 섞어서 과적합(overfit)을 방지하고 

multiprocessing(CPU_WORKER =>num_workers 관련)을 사용하여 데이터 검색 속도를 높이려고 한다.

(num_workers 참조 링크 : 

https://m.blog.naver.com/qbxlvnf11/221728476511

https://jybaek.tistory.com/799)

## Define Model Architecture (CNN 모델 만들기)

In [ ]:
class CNNRegressor(torch.nn.Module):
    def __init__(self):
        super(CNNRegressor, self).__init__()
        self.layer1 = torch.nn.Sequential(                           
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer4 = torch.nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.regressor = nn.Linear(3136,1)         #pytoch에서의 선형 회귀함수로, (입력의 차원, 출력의 차원)을 인수로 받음.


    def forward(self, x):   #순전파 알고리즘(역전파 X)
        # Simple CNN Model (Batch, 3, 128, 128 -> Batch, 64, 7, 7)
        # (Batch, 3, 128, 128)
        x = self.layer1(x)
        # (Batch, 8, 64, 64)
        x = self.layer2(x)
        # (Batch, 16, 32, 32)
        x = self.layer3(x)
        # (Batch, 32, 16, 16)
        x = self.layer4(x)
        # (Batch, 64, 7, 7) -> Flatten (Batch, 64*7*7(=3136))
        x = torch.flatten(x, start_dim=1)     
        # Regressor (Batch, 3136) -> (Batch, 1)
        out = self.regressor(x)
        return out

**nn.Sequential** :
forward() 함수의 형태를 Layer 형태로 나타낼 수 있게 하여 보다 가독성이 뛰어나게 코드 작성할 수 있도록 함


+Layer가 복잡해질수록 nn.Sequential의 효과가 뛰어남.

참조 : https://dororongju.tistory.com/147

## Train

In [ ]:
def train(model, optimizer, train_loader, vali_loader, scheduler, device):
    model.to(device)        #.to(device)를 통해 GPU 연산을 할 수 있게 한다.
    # Loss Function
    criterion = nn.L1Loss().to(device)       #loss함수 criterion을 L1Loss로 지정한다./.to(device)를 이용하여 GPU 연산한다.
    best_mae = 9999
    
    for epoch in range(1,CFG["EPOCHS"]+1):
        model.train()
        train_loss = []
        for img, label in tqdm(iter(train_loader)):
            img, label = img.float().to(device), label.float().to(device)
            
            optimizer.zero_grad()       #gradient를 0으로 초기화하기.기본적으로 gradient는 더해지기 때문에 중복 계산을 막기 위해 반복할 때(가중치 갱신 때)마다 0으로 설정함.

            # Data -> Model -> Output
            logit = model(img)
            # Calc loss
            loss = criterion(logit.squeeze(1), label)     #squeeze 함수 설명 아래 쪽에..

            # backpropagation
            loss.backward()     #loss 함수를 미분하여 gradient 계산
            optimizer.step()    #바로 윗 단계에서 수집된 gradient로 가중치(W,b) 최적화(업데이트 혹은 갱신)하기

            train_loss.append(loss.item())   #모델에서 계산된 loss 가 있다면, loss.item()을 통해 loss의 스칼라 값을 가져올 수 있음.
            
        if scheduler is not None:  #scheduler : If you don't call it, the learning rate won't be changed and stays at the initial value. 즉 learning rate를 조정하는 역할.
            scheduler.step()
            
        # Evaluation Validation set
        vali_mae = validation(model, vali_loader, criterion, device)
        
        print(f'Epoch [{epoch}] Train MAE : [{np.mean(train_loss):.5f}] Validation MAE : [{vali_mae:.5f}]\n')
        
        # Model Saved
        if best_mae > vali_mae:
            best_mae = vali_mae
            torch.save(model.state_dict(), './saved/best_model.pth')
            print('Model Saved.')

**squeeze() 함수 추가 설명**

(A x B x 1 x C x 1) 형태의 텐서에서

차원이 1인 부분을 제거하여 (A x B x C) 형태로 만들어 준다.

또한, 원하는 dimension 위치를 따로 선택하면, 해당 위치의 1만 삭제가 가능하다. *ex) logit.squeeze(dim=1)*

단, 해당 차원 위치의 size가 1이 아니라면, 삭제가 불가능하다.

참조 : 

https://wikidocs.net/55409

https://jimmy-ai.tistory.com/110 (pytorch squeeze 함수 정리)

In [ ]:
def validation(model, vali_loader, criterion, device):
    model.eval()  # Evaluation. 모델의 모든 Layer가 Evaluation-mode에 들어가게 해줌
    vali_loss = []
    with torch.no_grad():     #gradient를 계산해주는 Engiene을 비활성화 시켜 메모리를 줄여주고, 연산속도를 증가시킴.
        for img, label in tqdm(iter(vali_loader)):       #tqdm 라이브러리 : for 문의 in 구문을 tqdm으로 감싸기만 하면->작업진행률 바가 시각화 되어 나타남.
            img, label = img.float().to(device), label.float().to(device)

            logit = model(img)
            loss = criterion(logit.squeeze(1), label)   #위에서 선언했듯이 criterion=L1loss함수이다.
            
            vali_loss.append(loss.item())     #모델에서 계산된 loss 가 있다면, loss.item()을 통해 loss의 스칼라 값을 가져올 수 있음.

    vali_mae_loss = np.mean(vali_loss)
    return vali_mae_loss  #mae = mean absolute error(평균 절대 오차). mse와 거의 유사.

참조 : https://yuevelyne.tistory.com/10

## Run!!

In [ ]:
model = CNNRegressor().to(device)  #GPU를 사용한다

optimizer = torch.optim.SGD(params = model.parameters(), lr = CFG["LEARNING_RATE"])  #SGD optimizer를 사용한다
scheduler = None     #initial learning rate를 유지하여 사용한다

train(model, optimizer, train_loader, vali_loader, scheduler, device)

  0%|          | 0/160 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch [1] Train MAE : [63.01995] Validation MAE : [56.88789]

Model Saved.


  0%|          | 0/160 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch [9] Train MAE : [15.70300] Validation MAE : [36.53123]



  0%|          | 0/160 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch [10] Train MAE : [14.46853] Validation MAE : [16.36535]



## Inference

In [ ]:
def predict(model, test_loader, device):
    model.eval()
    model_pred = []
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.float().to(device)

            pred_logit = model(img)
            pred_logit = pred_logit.squeeze(1).detach().cpu()

            model_pred.extend(pred_logit.tolist())
    return model_pred

In [ ]:
test_dataset = CustomDataset(test_img_path, None, train_mode=False, transforms=test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# Validation Score가 가장 뛰어난 모델을 불러옵니다.
checkpoint = torch.load('./saved/best_model.pth')
model = CNNRegressor().to(device)
model.load_state_dict(checkpoint)

# Inference
preds = predict(model, test_loader, device)

  0%|          | 0/58 [00:00<?, ?it/s]

## Submission

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission['leaf_weight'] = preds
submission.to_csv('./submit.csv', index=False)